In [1]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations
AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]

In [2]:
dipep = [''.join(i) for i in itertools.product(AA, repeat = 2)]

In [3]:
arr = numpy.empty((0,402), dtype=numpy.float)
for record in SeqIO.parse('phixprot.faa', "fasta"):
        ll=len(record.seq)
        X = ProteinAnalysis(record.seq.__str__().replace('X','A').replace('J','L'))

        tt= [X.isoelectric_point(), X.instability_index()]
        tt_n = numpy.asarray(tt,dtype=numpy.float)

        dipep_count=[record.seq.count(i)/ll for i in dipep]
        dipep_count_n = numpy.asarray(dipep_count,dtype=numpy.float)
    
        cat_n= numpy.append(dipep_count_n,tt_n)
        cat_n = cat_n.reshape((1,cat_n.shape[0]))

        arr = numpy.append(arr,cat_n , axis=0)
        

In [4]:
import pickle
mean_arr=pickle.load(open( "dipep_new_mean.p", "rb" ) )
std_arr=pickle.load(open( "dipep_new_std.p", "rb" ) )

In [5]:
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        arr[i,j]=(arr[i,j]-mean_arr[j])/std_arr[j]

In [7]:
print(arr)

[[ 0.81462086 -0.32328229  1.35947221 ... -0.54030715 -0.63801846
  -1.93907445]
 [ 0.81462086 -0.32328229  1.35947221 ... -0.54030715 -0.57854736
  -1.98448114]
 [ 0.81462086 -0.32328229  1.35947221 ... -0.54030715 -0.57854736
  -1.98448114]
 [ 0.81462086 -0.32328229  1.35947221 ... -0.54030715 -0.57854736
  -1.98738009]
 [ 0.81462086 -0.32328229  1.35947221 ... -0.54030715 -0.57854736
  -1.98448114]
 [ 0.81462086 -0.32328229  1.35947221 ... -0.54030715 -0.57854736
  -1.98448114]]


In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import load_model

model = load_model('di_new_model.h5')

In [11]:
print(model.predict_classes(arr))

[1 1 1 1 1 1]


In [12]:
model.predict(arr)

array([[4.6981447e-17, 1.0000000e+00, 7.9821674e-14, 2.3182189e-19,
        6.1451313e-12, 4.7174423e-13, 2.0969488e-13, 1.8793634e-20,
        6.9035241e-18, 1.6074851e-14],
       [4.3804057e-17, 1.0000000e+00, 2.5448873e-14, 7.1206202e-20,
        1.5765094e-12, 4.4286501e-14, 1.2330572e-13, 5.3764203e-21,
        6.8350229e-19, 2.2250554e-15],
       [4.3039475e-17, 1.0000000e+00, 2.2454996e-14, 3.0069970e-20,
        1.0336970e-12, 4.3424205e-14, 9.7145883e-14, 4.5927002e-21,
        4.0798220e-19, 1.7001231e-15],
       [1.5797028e-16, 1.0000000e+00, 1.3425029e-13, 3.0671836e-19,
        7.4303359e-12, 4.3568481e-14, 5.9913375e-13, 1.9525075e-20,
        7.6705641e-18, 8.5637089e-15],
       [4.4936183e-16, 1.0000000e+00, 4.3188055e-14, 1.1769109e-19,
        6.6282691e-12, 7.4518435e-14, 1.0823283e-12, 1.0316345e-20,
        2.6257269e-17, 7.4743314e-15],
       [4.3804057e-17, 1.0000000e+00, 2.5448873e-14, 7.1206202e-20,
        1.5765094e-12, 4.4286501e-14, 1.2330572e-13, 5.37